In [97]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

import glob
import os

In [100]:
input_file = r'C:\SSAFY\bigdata\gamedata' # csv파일들이 있는 디렉토리 위치
output_file = r'C:\SSAFY\bigdata\gamedata\steamgame.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'onlyapp_*')) # glob함수로 sales_로 시작하는 파일들을 모은다
print(allFile_list)
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    md = pd.read_csv(file) # for구문으로 csv파일들을 읽어 들인다
    allData.append(md) # 빈 리스트에 읽어 들인 내용을 추가한다

dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용을 병합
# axis=0은 수직으로 병합함. axis=1은 수평. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인데스를 빼려면 False로 설정

['C:\\SSAFY\\bigdata\\gamedata\\onlyapp_games_1.csv', 'C:\\SSAFY\\bigdata\\gamedata\\onlyapp_games_2.csv', 'C:\\SSAFY\\bigdata\\gamedata\\onlyapp_games_3.csv']


In [101]:
md = pd.read_csv('C:/SSAFY/bigdata/gamedata/steamgame.csv')
md.head()

,_id,url,name,desc_snippet,recent_reviews_count,recent_reviews_percent,all_reviews_count,all_reviews_percent,release_date,developer,...,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
0,379720.0,https://store.steampowered.com/app/379720/DOOM/,DOOM,Now includes all three premium DLC packs (Unto...,554.0,89.0,42550.0,92.0,"May 12, 2016",id Software,...,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99
1,578080.0,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,6214.0,49.0,836608.0,49.0,"Dec 21, 2017",PUBG Corporation,...,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99
2,637090.0,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Take command of your own mercenary outfit of '...,166.0,54.0,7030.0,71.0,"Apr 24, 2018",Harebrained Schemes,...,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99
3,221100.0,https://store.steampowered.com/app/221100/DayZ/,DayZ,The post-soviet country of Chernarus is struck...,932.0,57.0,167115.0,61.0,"Dec 13, 2018",Bohemia Interactive,...,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99
4,8500.0,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,EVE Online is a community-driven spaceship MMO...,287.0,54.0,11481.0,74.0,"May 6, 2003",CCP,...,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free


In [102]:
all_reviews_counts = md[md['all_reviews_count'].notnull()]['all_reviews_count'].astype('int')
recent_reviews_counts = md[md['recent_reviews_count'].notnull()]['recent_reviews_count'].astype('int')
all_scores =md[md['all_reviews_percent'].notnull()]['all_reviews_percent'].astype('int')
recent_scores =md[md['recent_reviews_percent'].notnull()]['recent_reviews_percent'].astype('int')
C1 = all_scores.mean()
C2 = recent_scores.mean()
print(C1)
print(C2)

70.56218332565638
71.40154867256638


In [103]:
m1 = all_reviews_counts.quantile(0.95)
m2 = recent_reviews_counts.quantile(0.95)
print(m1)
print(m2)

2008.0
377.4499999999998


In [104]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
print(md['year'])

0        2016
1        2017
2        2018
3        2018
4        2003
         ... 
38020    2019
38021    2019
38022    2018
38023    2018
38024    2018
Name: year, Length: 38025, dtype: object


In [105]:
qualified1 = md[(md['all_reviews_count'] >= m1) & (md['all_reviews_count'].notnull()) & (md['all_reviews_percent'].notnull())][['name', 'year', 'all_reviews_count', 'all_reviews_percent', 'popular_tags']]
qualified1['all_reviews_count'] = qualified1['all_reviews_count'].astype('int')
qualified1['all_reviews_percent'] = qualified1['all_reviews_percent'].astype('int')
print(qualified1.shape)

qualified2 = md[(md['recent_reviews_count'] >= m2) & (md['recent_reviews_count'].notnull()) & (md['recent_reviews_percent'].notnull())][['name', 'year', 'recent_reviews_count', 'recent_reviews_percent', 'popular_tags']]
qualified2['recent_reviews_count'] = qualified2['recent_reviews_count'].astype('int')
qualified2['recent_reviews_percent'] = qualified2['recent_reviews_percent'].astype('int')
print(qualified2.shape)

(1426, 5)
(136, 5)


In [106]:
def all_weighted_rating(x):
    v = x['all_reviews_count']
    R = x['all_reviews_percent']
    return (v/(v+m1) * R) + (m1/(m1+v) * C1)

In [107]:
def recent_weighted_rating(x):
    v = x['recent_reviews_count']
    R = x['recent_reviews_percent']
    return (v/(v+m2) * R) + (m2/(m2+v) * C2)

In [108]:
qualified1['wr'] = qualified1.apply(all_weighted_rating, axis = 1)

In [109]:
qualified2['wr'] = qualified2.apply(recent_weighted_rating, axis = 1)

In [110]:
qualified1 = qualified1.sort_values('wr', ascending = False).head(250)

In [111]:
qualified2 = qualified2.sort_values('wr', ascending = False).head(250)

In [75]:
qualified1.head(15)

,name,year,all_reviews_count,all_reviews_percent,popular_tags,wr
5359,Sid Meier's Civilization V: Brave New World,2013,1147,95,"Strategy,Turn-Based,Turn-Based Strategy,4X,Mul...",90.166693
3143,Tiny and Big: Grandpa's Leftovers,2012,2535,92,"Indie,Physics,Platformer,Puzzle,Adventure,3D P...",89.828057
6510,KARAKARA,2016,789,96,"Visual Novel,Anime,Sexual Content,Nudity,Cute,...",89.121938
5364,Rochard,2011,891,91,"Platformer,Indie,Action,Puzzle,Physics,Sci-fi,...",85.379727
11667,Yoku's Island Express,2018,469,96,"Adventure,Pinball,Metroidvania,Indie,Platforme...",85.333677
3523,Teslagrad,2013,1256,85,"Platformer,Indie,Puzzle,Metroidvania,2D,Steamp...",81.370516
2911,King's Bounty: Armored Princess,2009,437,89,"RPG,Turn-Based Strategy,Strategy,Fantasy,Femal...",79.140011
578,"Precipice of Darkness, Episode One",2008,279,93,"RPG,Adventure,Comedy,Funny,Singleplayer,Turn-B...",78.009888
5288,Vector,2013,448,87,"Parkour,Action,Indie,Platformer,Singleplayer,2...",77.733690
5670,War of the Human Tanks,2012,306,91,"Anime,Strategy,Indie,Adventure,Visual Novel,Mi...",77.514034


In [113]:
qualified2.head(15)

,name,year,recent_reviews_count,recent_reviews_percent,popular_tags,wr
235,Left 4 Dead 2,2009,3266,96,"Zombies,Co-op,FPS,Multiplayer,Shooter,Action,O...",93.451678
367,The Witcher® 3: Wild Hunt,2015,2386,96,"Open World,RPG,Story Rich,Atmospheric,Mature,F...",92.640183
323,Terraria,2011,2744,95,"Sandbox,Survival,2D,Adventure,Multiplayer,Craf...",92.146443
297,Garry's Mod,2006,3125,94,"Sandbox,Multiplayer,Funny,Moddable,Building,Co...",91.564623
237,Katana ZERO,2019,1396,97,"Action,Pixel Graphics,Great Soundtrack,Cyberpu...",91.551786
250,Wallpaper Engine,2018,1455,96,"Utilities,Software,Design & Illustration,Anima...",90.933185
305,Don't Starve Together,2016,1860,94,"Survival,Multiplayer,Co-op,Open World,Adventur...",90.187720
221,Portal 2,2011,899,98,"Puzzle,Co-op,First-Person,Sci-fi,Comedy,Single...",90.134760
166,Euro Truck Simulator 2,2012,2036,93,"Simulation,Driving,Open World,Realistic,Relaxi...",89.622124
1664,Doki Doki Literature Club!,2017,1237,95,"Psychological Horror,Anime,Visual Novel,Free t...",89.482805


In [115]:
s = md.apply(lambda x: pd.Series(x['popular_tags']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'popular_tags'
tags_md = md.drop('popular_tags', axis=1).join(s)

In [144]:
def build_chart(popular_tags, percentile=0.85):
    df = tags_md[tags_md['popular_tags'].str.contains(popular_tags, na = True)]
    all_reviews_counts = df[df['all_reviews_count'].notnull()]['all_reviews_count'].astype('int')
    all_reviews_percents = df[df['all_reviews_percent'].notnull()]['all_reviews_percent'].astype('int')
    C = all_reviews_percents.mean()
    m = all_reviews_counts.quantile(percentile)
    
    qualified = df[(df['all_reviews_count'] >= m) & (df['all_reviews_count'].notnull()) & (df['all_reviews_percent'].notnull())][['name', 'year', 'all_reviews_count', 'all_reviews_percent']]
    qualified['all_reviews_count'] = qualified['all_reviews_count'].astype('int')
    qualified['all_reviews_percent'] = qualified['all_reviews_percent'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['all_reviews_count']/(x['all_reviews_count'] + m) * x['all_reviews_percent']) + (m/(m+x['all_reviews_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [146]:
build_chart('Indie').head(15)

,name,year,all_reviews_count,all_reviews_percent,wr
294,Factorio,2016,38652,98,97.805404
5698,Kung Fury,2015,17215,98,97.566975
3194,Iron Snout,2016,14306,98,97.480619
2750,One Finger Death Punch,2014,11134,98,97.336294
205,The Binding of Isaac: Rebirth,2014,63149,97,96.884988
194,Mount & Blade: Warband,2010,59100,97,96.877146
2170,Finding Paradise,2017,6307,98,96.849999
526,Hotline Miami,2012,38402,97,96.811410
671,BattleBlock Theater®,2014,35947,97,96.798631
331,RimWorld,2018,31754,97,96.772274


In [151]:
smd = md
smd.shape

(38025, 22)

In [153]:
smd['popular_tags'] = smd['popular_tags'].fillna('')
smd['description'] = smd['game_description'] + smd['popular_tags']
smd['game_description'] = smd['game_description'].fillna('')

In [156]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [158]:
tfidf_matrix.shape

(38025, 2402319)

In [160]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [162]:
cosine_sim[0]

array([1.00000000e+00, 3.24056867e-02, 1.88164554e-02, ...,
       1.53291590e-02, 4.22454110e-04, 1.82935079e-02])

In [168]:
smd = smd.reset_index()
names = smd['name']
indices = pd.Series(smd.index, index=smd['name'])

ValueError: cannot insert level_0, already exists

In [169]:
def get_recommendations(name):
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    game_indices = [i[0] for i in sim_scores]
    return names.iloc[game_indices]

In [174]:
get_recommendations('Stardew Valley').head(10)

227                                               Forager
5588                                       My Free Farm 2
670                                           Garden Paws
760                                         Farm Together
18603                                Harvest Simulator VR
21989                               Farming 6-in-1 bundle
2372                                         Harvest Life
37643    Stranded Sails - Explorers of the Cursed Islands
1096                                     Farmer's Dynasty
17499                                         Little Farm
Name: name, dtype: object

In [188]:
smd['soup'] = smd['genre'] + smd['developer'] + smd['publisher'] + smd['popular_tags']
smd['soup'] = smd['soup'].fillna('')

In [190]:
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [192]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [198]:
# smd = smd.reset_index()
names = smd['name']
indices = pd.Series(smd.index, index=smd['name'])

In [200]:
get_recommendations('Stardew Valley').head(10)

37485                                      Circadian City
1117                             Cattails | Become a Cat!
5429                                         The Islander
4126                                             Plantera
17017                           De'Vine: World of Shadows
25762                                    The Space Garden
3548                                        Feel The Snow
37643    Stranded Sails - Explorers of the Cursed Islands
6338                                       Monster Garden
7402                                        Creepy Castle
Name: name, dtype: object

In [204]:
def improved_recommendations(name):
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    game_indices = [i[0] for i in sim_scores]
    
    games = smd.iloc[game_indices][['name', 'all_reviews_count', 'all_reviews_percent', 'year']]
    all_reviews_counts = games[games['all_reviews_count'].notnull()]['all_reviews_count'].astype('int')
    all_reviews_percents = games[games['all_reviews_percent'].notnull()]['all_reviews_percent'].astype('int')
    C = all_reviews_percents.mean()
    m = all_reviews_counts.quantile(0.60)
    qualified = games[(games['all_reviews_count'] >= m) & (games['all_reviews_count'].notnull()) & (games['all_reviews_percent'].notnull())]
    qualified['all_reviews_count'] = qualified['all_reviews_count'].astype('int')
    qualified['all_reviews_percent'] = qualified['all_reviews_percent'].astype('int')
    qualified['wr'] = qualified.apply(all_weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [208]:
improved_recommendations('Among Us')

,name,all_reviews_count,all_reviews_percent,year,wr
187,Just Shapes & Beats,2124,96,2018,83.638157
1954,Awesomenauts - the 2D moba,24481,84,2012,82.981346
298,Brawlhalla,90153,83,2017,82.729005
7656,Sonic the Hedgehog 4 - Episode II,610,80,2012,72.761216
669,Super Mega Baseball 2,446,82,2018,72.640939
3165,FORCED: Slightly Better Edition,1647,75,2013,72.561933
3680,PixelJunk™ Monsters Ultimate,302,81,2013,71.926781
20811,Sacred Citadel,437,72,2013,70.819167
1296,Hades' Star,375,70,2019,70.473716
24717,Street Fighter X Tekken,1222,56,2012,65.052899
